## Example of a Neural Network Pipeline with Pytorch

In [1]:
# Import all necessary modules
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import os
import shutil
import datetime

In [2]:
# Simple Vision Transformer (ViT) implementation
class SimpleViT(nn.Module):
    def __init__(self, image_size=28, patch_size=7, num_classes=10, dim=64, depth=4, heads=4, mlp_dim=128):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = patch_size * patch_size

        self.patch_size = patch_size
        self.dim = dim
        self.patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim),
            num_layers=depth
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.reshape(B, C, H // self.patch_size, self.patch_size, W // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 1, 3, 5).reshape(B, -1, C * self.patch_size * self.patch_size)
        x = self.patch_embedding(x)
        x = x + self.pos_embedding
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.mlp_head(x)


In [3]:
# Simple Swin Transformer implementation
class SimpleSwinTransformer(nn.Module):
    def __init__(self, image_size=28, patch_size=7, num_classes=10, dim=64, depth=4, heads=4, mlp_dim=128):
        super().__init__()
        assert image_size % patch_size == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = patch_size * patch_size

        self.patch_size = patch_size
        self.dim = dim
        self.patch_embedding = nn.Linear(patch_dim, dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches, dim))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads, dim_feedforward=mlp_dim),
            num_layers=depth
        )
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.reshape(B, C, H // self.patch_size, self.patch_size, W // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 1, 3, 5).reshape(B, -1, C * self.patch_size * self.patch_size)
        x = self.patch_embedding(x)
        x = x + self.pos_embedding
        x = self.transformer(x)
        x = x.mean(dim=1)
        return self.mlp_head(x)

In [ ]:
# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# check of run path exists and create if not
if os.path.exists('runs'):
    print("✅ 'runs' directory already exists. Using existing logs directory...")
else:
    print("❗ 'runs' directory does not exist. Creating new logs directory...")
    os.makedirs('runs', exist_ok=True)

❗ 'runs' directory already exists. Using existing logs directory...


In [5]:
# Model setup
#model = SimpleViT() # or SimpleSwinTransformer()
model = SimpleSwinTransformer() # or SimpleViT()
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

C:\Users\Charon\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [6]:
# TensorBoard writer - use default runs directory
try:
    writer = SummaryWriter()
    print("✅ TensorBoard writer created successfully!")
    print(f"Logging to: {writer.log_dir}")
except Exception as e:
    print(f"❌ Error creating TensorBoard writer: {e}")
    # Fallback: create manual logs
    import tempfile
    temp_dir = tempfile.mkdtemp(prefix='tensorboard_')
    writer = SummaryWriter(temp_dir)
    print(f"✅ Using temporary directory: {temp_dir}")

# Training loop with logging
print("\n🏋️ Starting training...")
model.train()

for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    # Log every 10 batches
    if batch_idx % 10 == 0:
        writer.add_scalar('Loss/Train', loss.item(), batch_idx)
        print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")
    
    if batch_idx >= 100:  # limit for demo
        break

# Log final metrics
writer.add_scalar('Final/TrainingLoss', loss.item(), 0)
writer.close()

print(f"\n✅ Training complete!")
print(f"📊 TensorBoard logs saved to: {writer.log_dir}")
print("🌐 Now run the TensorBoard launcher below to visualize the results!")

✅ TensorBoard writer created successfully!
Logging to: runs\Aug05_17-00-45_Pluto

🏋️ Starting training...
Batch 0, Loss: 2.5291
Batch 0, Loss: 2.5291
Batch 10, Loss: 2.3125
Batch 10, Loss: 2.3125
Batch 20, Loss: 2.3138
Batch 20, Loss: 2.3138
Batch 30, Loss: 2.2914
Batch 30, Loss: 2.2914
Batch 40, Loss: 2.3098
Batch 40, Loss: 2.3098
Batch 50, Loss: 2.3000
Batch 50, Loss: 2.3000
Batch 60, Loss: 2.3218
Batch 60, Loss: 2.3218
Batch 70, Loss: 2.2674
Batch 70, Loss: 2.2674
Batch 80, Loss: 2.2345
Batch 80, Loss: 2.2345
Batch 90, Loss: 2.0386
Batch 90, Loss: 2.0386
Batch 100, Loss: 1.8554

✅ Training complete!
📊 TensorBoard logs saved to: runs\Aug05_17-00-45_Pluto
🌐 Now run the TensorBoard launcher below to visualize the results!
Batch 100, Loss: 1.8554

✅ Training complete!
📊 TensorBoard logs saved to: runs\Aug05_17-00-45_Pluto
🌐 Now run the TensorBoard launcher below to visualize the results!


In [ ]:
# Check if tensorboard is installed, if not install it
try:
    from torch.utils.tensorboard import SummaryWriter
    print("✅ TensorBoard is available!")
except ImportError:
    print("Tensorboard is not installed. Installing now...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorboard'])
    from torch.utils.tensorboard import SummaryWriter
    print("✅ TensorBoard installed successfully!")

✅ TensorBoard is available!

📋 Next steps:
1. Make sure you have run the training loop (cell 6)
2. Use one of the TensorBoard launcher cells below
3. Open http://localhost:6006 in your browser

💡 Note: On Windows, you need to run TensorBoard via Python module, not directly from command line


In [8]:
# Launch TensorBoard from within the notebook
import subprocess
import sys
import os
import threading
import time

def launch_tensorboard():
    """Launch TensorBoard in a separate thread"""
    try:
        # Get the current directory
        current_dir = os.getcwd()
        runs_dir = os.path.join(current_dir, 'runs')
        
        # Check if runs directory exists
        if not os.path.exists(runs_dir):
            print(f"Warning: {runs_dir} directory not found. Make sure you have run the training loop first.")
            return
        
        print(f"Starting TensorBoard...")
        print(f"Log directory: {runs_dir}")
        print("TensorBoard will be available at: http://localhost:6006")
        print("Press Ctrl+C in the terminal to stop TensorBoard")
        
        # Launch TensorBoard using Python module
        cmd = [sys.executable, '-m', 'tensorboard.main', '--logdir', runs_dir, '--port', '6006']
        process = subprocess.run(cmd, check=True)
        
    except subprocess.CalledProcessError as e:
        print(f"Error starting TensorBoard: {e}")
    except KeyboardInterrupt:
        print("\nTensorBoard stopped by user")

# Start TensorBoard in a separate thread so it doesn't block the notebook
tensorboard_thread = threading.Thread(target=launch_tensorboard, daemon=True)
tensorboard_thread.start()

# Give it a moment to start
time.sleep(2)
print("\nIf TensorBoard started successfully, you can access it at:")
print("http://localhost:6006")
print("\nTo stop TensorBoard, restart the kernel or use Ctrl+C in the terminal.")

Starting TensorBoard...
Log directory: c:\Users\Charon\Unterlagen\Uni\Master\Faecher\RKIM_FuE_KI_Pipeline\Test_Software\runs
TensorBoard will be available at: http://localhost:6006
Press Ctrl+C in the terminal to stop TensorBoard

If TensorBoard started successfully, you can access it at:
http://localhost:6006

To stop TensorBoard, restart the kernel or use Ctrl+C in the terminal.

If TensorBoard started successfully, you can access it at:
http://localhost:6006

To stop TensorBoard, restart the kernel or use Ctrl+C in the terminal.


## 📊 TensorBoard Anleitung

### Was ist TensorBoard?
TensorBoard ist ein Visualisierungstool für Machine Learning Experimente. Es hilft dir dabei:
- **Trainingsverläufe** zu visualisieren (Loss, Accuracy, etc.)
- **Modellarchitekturen** zu betrachten
- **Histogramme** von Gewichten und Gradienten zu analysieren
- **Bilder und Embeddings** zu visualisieren

### 🚀 Wie verwendest du TensorBoard?

#### Schritt 1: Training ausführen
Stelle sicher, dass du die Trainingszellen ausgeführt hast. Diese erstellen die Log-Dateien im `runs/` Ordner.

#### Schritt 2: TensorBoard öffnen
- Öffne deinen Browser
- Gehe zu: `http://localhost:6006`
- Du siehst jetzt deine Trainingsgraphen!

### 🔍 Was siehst du in TensorBoard?
- **SCALARS Tab**: Hier siehst du den Trainingsloss über die Zeit
- **GRAPHS Tab**: Hier kannst du die Modellarchitektur visualisieren
- **DISTRIBUTIONS/HISTOGRAMS**: Gewichtsverteilungen (falls geloggt)

2. **SCALARS Tab**: 
   - Du siehst einen Graphen namens "Loss/Train"
   - Dieser zeigt, wie der Trainingsloss über die 100 Batches sinkt (von ~2.5 auf ~2.0)
   - Du siehst auch "Final/TrainingLoss" mit dem finalen Loss-Wert

3. **Was die Graphen bedeuten**:
   - X-Achse: Batch-Nummer (0 bis 100)
   - Y-Achse: Loss-Wert
   - Der Graph sollte eine fallende Tendenz zeigen → das Modell lernt! 📈

   ### 💡 Tipps
- TensorBoard aktualisiert sich automatisch, wenn neue Daten hinzugefügt werden
- Du kannst mehrere Experimente vergleichen, indem du verschiedene Ordner in `runs/` erstellst
- Verwende aussagekräftige Namen für deine Logs: `writer = SummaryWriter('runs/experiment_1')`